Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

# CSE 30 Fall 2021 - Homework 13

## Sudoku


### Instructions

Please disregard the YOUR NAME and COLLABORATORS above.  They are put there atomatically by the grading tool.
You can find instructions on how to work on a homework on Canvas.  Here is a short summary: 

### Submitting your work

To submit your work: 

* First, click on "Runtime > Restart and run all", and check that you get no errors.  This enables you to catch any error you might have introduced, and not noticed, due to your running cells out of order. 
* Second, download the notebook in .ipynb format (File > Download .ipynb) and upload the .ipynb file to [this form](https://docs.google.com/forms/d/e/1FAIpQLSeK1VhGtlPNj6D7UoCgl1VhGJHsk7Rx7UxUrEpAcDnR7PRDuA/viewform?usp=sf_link). 

You can submit multiple times; the last submission before the deadline is the one that counts.

### Homework format

For each question in this notebook, there is: 

* A text description of the problem. 
* One or more places where you have to insert your solution.  You need to complete every place marked: 

    `# YOUR CODE HERE`
    
    and you should not modify any other place. 
* One or more test cells.  Each cell is worth some number of points, marked at the top.  You should not modify these tests cells.  The tests pass if no error is printed out: when there is a statement that says, for instance: 

    `assert x == 2`
    
    then the test passes if `x` has value 2, and fails otherwise.  You can insert a `print(x)` (for this case!) somewhere if you want to debug your work; it is up to you.  
    
### Notes:

* Your code will be tested both according to the tests you can see (the `assert` statements you can see), _and_ additional tests.  This prevents you from hard-coding the answer to the particular questions posed.  Your code should solve the _general_ intended case, not hard-code the particular answer for the values used in the tests. 

* **Please do not delete or add cells!** The test is autograded, and if you modify the test by adding or deleting cells, even if you re-add cells you delete, you may not receive credit. 

* **Please do not import modules that are not part of the [standard library](https://docs.python.org/3/library/index.html).** You do not need any, and they will likely not available in the grading environment, leading your code to fail. 

* **If you are inactive too long, your notebook might get disconnected from the back-end.** Your work is never lost, but you have to re-run all the cells before you continue. 

* You can write out print statements in your code, to help you test/debug it. But remember: the code is graded on the basis of what it outputs or returns, not on the basis of what it prints.

* **TAs and tutors have access to this notebook,** so if you let them know you need their help, they can look at your work and give you advice. 

### Grading

Each cell where there are tests is worth a certain number of points.  You get the points allocated to a cell only if you pass _all_ the tests in the cell. 

The tests in a cell include both the tests you can see, and other, similar, tests that are used for grading only.  Therefore, you cannot hard-code the solutions: you really have to solve the essence of the problem, to receive the points in a cell. 

### Code of Conduct

* Work on the test yourself, alone. 
* You can search documentation on the web, on sites such as the Python documentation sites, Stackoverflow, and similar, and you can use the results. 
* You cannot share your work with others or solicit their help.


## Submission

[Please submit to this Google Form](https://docs.google.com/forms/d/e/1FAIpQLSf0r9gB2y6Wcf4pWM9aqpzyZfQOvNqFJol36NtS4Msdvd3VXA/viewform?usp=sf_link).

Deadline: Tuesday December 3, 11pm (check on Canvas for updated information).

## Test Format

There are 6 questions, but not all of them require you to write code: 
* Question 1 asks you to complete the code for propagating a single cell.
* Question 2 asks you to write the code to propagate all cells. 
* For Question 3, you just need to copy your solution to Question 2 in another place. 
* Question 4 asks you to implement a helper function that detects elements that occur in only one of a list of sets. 
* Question 5 asks you to implement the _where can it go_ heuristic. 
* Question 6 simply checks that your code is efficient; you don't need to write any code. 

There are a total of 70 points. 

Let us write a [Sudoku](https://en.wikipedia.org/wiki/Sudoku) solver.  We want to get as input a Sudoku with some cells filled with values, and we want to get as output a solution, if one exists, and otherwise a notice that the input Sudoku puzzle has no solutions. 

You will wonder, why spend so much time on Sudoku? 

For two reasons. 

First, the way we go about solving Sudoku is prototypical of a very large number of problems in computer science.  In these problems, the solution is attained through a mix of search (we attempt to fill a square with a number and see if it works out), and constraint propagation (if we fill a square with, say, a 1, then there can be no 1's in the same row, column, and 3x3 square).

Second, and related, the way we go about solving Sudoku puzzles is closely related to how [SAT solvers](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem#Algorithms_for_solving_SAT) work.  So closely related, in fact, that while _we_ describe for you how a Sudoku solver works, _you_ will have to write a SAT solver as exercise. 


## Sudoku representation

First, let us do some grunt work and define a representation for a Sudoku problem. 

One initial idea would be to represent a Sudoku problem via a $9 \times 9$ matrix, where each entry can be either a digit from 1 to 9, or 0 to signify "blank".  This would work in some sense, but it would not be a very useful representation.  If you have solved Sudoku by hand (and if you have not, please go and solve a couple; it will teach you a lot about what we need to do), you will know that the following strategy works: 

Repeat: 
* Look at all blank spaces.  Can you find one where only one digit fits? If so, write the digit there. 
* If you cannot find any blank space as above, try to find one where only a couple or so digits can fit.  Try putting in one of those digits, and see if you can solve the puzzle with that choice.  If not, backtrack, and try another digit. 

Thus, it will be very useful to us to remember not only the known digits, but also, which digits can fit into any blank space. 
Hence, we represent a Sudoku problem via a $9 \times 9$ matrix of _sets_: each set contains the digits that can fit in a given space. 
Of course, a known digit is just a set containing only one element. 
We will solve a Sudoku problem by progressively "shrinking" these sets of possibilities, until they all contain exactly one element. 

Let us write some code that enables us to define a Sudoku problem, and display it for us; this will be very useful both for our fun and for debugging. 


First, though, let's write a tiny helper function that returns the only element from a singleton set.

In [2]:
def getel(s):
    """Returns the unique element in a singleton set (or list)."""
    assert len(s) == 1
    return list(s)[0]


In [3]:
import json

class Sudoku(object):

    def __init__(self, elements):
        """Elements can be one of:
        Case 1: a list of 9 strings of length 9 each.
        Each string represents a row of the initial Sudoku puzzle,
        with either a digit 1..9 in it, or with a blank or _ to signify
        a blank cell.
        Case 2: an instance of Sudoku.  In that case, we initialize an
        object to be equal (a copy) of the one in elements.
        Case 3: a list of list of sets, used to initialize the problem."""
        if isinstance(elements, Sudoku):
            # We let self.m consist of copies of each set in elements.m
            self.m = [[x.copy() for x in row] for row in elements.m]
        else:
            assert len(elements) == 9
            for s in elements:
                assert len(s) == 9
            # We let self.m be our Sudoku problem, a 9x9 matrix of sets.
            self.m = []
            for s in elements:
                row = []
                for c in s:
                    if isinstance(c, str):
                        if c.isdigit():
                            row.append({int(c)})
                        else:
                            row.append({1, 2, 3, 4, 5, 6, 7, 8, 9})
                    else:
                        assert isinstance(c, set)
                        row.append(c)
                self.m.append(row)


    def show(self, details=False):
        """Prints out the Sudoku matrix.  If details=False, we print out
        the digits only for cells that have singleton sets (where only
        one digit can fit).  If details=True, for each cell, we display the
        sets associated with the cell."""
        if details:
            print("+-----------------------------+-----------------------------+-----------------------------+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    # We represent the set {2, 3, 5} via _23_5____
                    s = ''
                    for k in range(1, 10):
                        s += str(k) if k in self.m[i][j] else '_'
                    r += s
                    r += '|' if (j + 1) % 3 == 0 else ' '
                print(r)
                if (i + 1) % 3 == 0:
                    print("+-----------------------------+-----------------------------+-----------------------------+")
        else:
            print("+---+---+---+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    if len(self.m[i][j]) == 1:
                        r += str(getel(self.m[i][j]))
                    else:
                        r += "."
                    if (j + 1) % 3 == 0:
                        r += "|"
                print(r)
                if (i + 1) % 3 == 0:
                    print("+---+---+---+")


    def to_string(self):
        """This method is useful for producing a representation that
        can be used in testing."""
        as_lists = [[list(self.m[i][j]) for j in range(9)] for i in range(9)]
        return json.dumps(as_lists)


    @staticmethod
    def from_string(s):
        """Inverse of above."""
        as_lists = json.loads(s)
        as_sets = [[set(el) for el in row] for row in as_lists]
        return Sudoku(as_sets)


    def __eq__(self, other):
        """Useful for testing."""
        return self.m == other.m


Let us input a problem (the Sudoku example found on [this Wikipedia page](https://en.wikipedia.org/wiki/Sudoku)) and check that our serialization and deserialization works.

In [4]:
# Let us ensure that nose is installed.
try:
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal


In [5]:
from nose.tools import assert_equal

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.show(details=True)
s = sd.to_string()
sdd = Sudoku.from_string(s)
sdd.show(details=True)
assert_equal(sd, sdd)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+---------------------

Let's test our constructor statement when passed a Sudoku instance.

In [6]:
sd1 = Sudoku(sd)
assert_equal(sd, sd1)


## Constraint propagation

When the set in a Sudoku cell contains only one element, this means that the digit at that cell is known. 
We can then propagate the knowledge, ruling out that digit in the same row, in the same column, and in the same 3x3 cell. 

We first write a method that propagates the constraint from a single cell.  The method will return the list of newly-determined cells, that is, the list of cells who also now (but not before) are associated with a 1-element set.  This is useful, because we can then propagate the constraints from those cells in turn.  Further, if an empty set is ever generated, we raise the exception Unsolvable: this means that there is no solution to the proposed Sudoku puzzle. 

We don't want to steal all the fun from you; thus, we will give you the main pieces of the implemenetation, but we ask you to fill in the blanks.  We provide tests so you can catch any errors right away.

## Question 1: Propagating a single cell

In [7]:
class Unsolvable(Exception):
    pass


def sudoku_ruleout(self, i, j, x):
    """The input consists in a cell (i, j), and a value x.
    The function removes x from the set self.m[i][j] at the cell, if present, and:
    - if the result is empty, raises Unsolvable;
    - if the cell used to be a non-singleton cell and is now a singleton
      cell, then returns the set {(i, j)};
    - otherwise, returns the empty set."""
    c = self.m[i][j]
    n = len(c)
    c.discard(x)
    self.m[i][j] = c
    if len(c) == 0:
        raise Unsolvable()
    return {(i, j)} if 1 == len(c) < n else set()

Sudoku.ruleout = sudoku_ruleout


The method `propagate_cell(ij)` takes as input a pair `ij` of coordinates.  If the set of possible digits `self.m[i][j]` for cell i,j contains more than one digit, then no propagation is done.  If the set contains a single digit `x`, then we: 

* Remove `x` from the sets of all other cells on the same row, column, and 3x3 block. 
* Collect all the newly singleton cells that are formed, due to the digit `x` being removed, and we return them as a set. 

We give you an implementation that takes care of removing `x` from the same row, and we ask you to complete the implementation to take care of the column and 3x3 block as well.

In [8]:
### Exercise: define cell propagation

def sudoku_propagate_cell(self, ij):
    """Propagates the singleton value at cell (i, j), returning the list
    of newly-singleton cells."""
    i, j = ij
    if len(self.m[i][j]) > 1:
        # Nothing to propagate from cell (i,j).
        return set()
    # We keep track of the newly-singleton cells.
    newly_singleton = set()
    x = getel(self.m[i][j]) # Value at (i, j).
    # Same row.
    for jj in range(9):
        if jj != j: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(i, jj, x))
        
    # Same column.
    # YOUR CODE HERE
    for ii in range(9):
        if ii != i: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(ii, j, x))
    # Same block of 3x3 cells.
    # YOUR CODE HERE
    min = {0:0,1:-1,2:-2}
    max = {0:3,1:2,2:1}
    for ii in range(i//3*3,i//3*3+3):
      for jj in range(j//3*3,j//3*3+3):
        if ii != i and jj != j:
          newly_singleton.update(self.ruleout(ii, jj, x))
    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.propagate_cell = sudoku_propagate_cell


In [9]:
# Here you can write your own tests if you like. 

# YOUR CODE HERE

In [10]:
### 10 points: Tests for cell propagation

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
try:
    tsd.propagate_cell((0, 2))
except Unsolvable:
    print("Good! It was unsolvable.")
else:
    raise Exception("Hey, it was unsolvable")

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2, 3]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)

assert_equal(tsd.propagate_cell((0, 2)), {(0, 8), (2, 0)})



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ _2_______|_____6___ ______7__ _______8_|________9 12_4_____ _2_______|
|_____6___ ______7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _______8_|
|12_______ ________9 _______8_|__3______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 _______8_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

### Propagating all cells, once

The simplest thing we can do is propagate each cell, once. 

In [11]:
def sudoku_propagate_all_cells_once(self):
    """This function propagates the constraints from all singletons."""
    for i in range(9):
        for j in range(9):
            self.propagate_cell((i, j))

Sudoku.propagate_all_cells_once = sudoku_propagate_all_cells_once


In [12]:
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.propagate_all_cells_once()
sd.show()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|853|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|..7|284|
|...|419|.35|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_2___6___ ______7__ _2_4_6_8_|1__4___89 12_4____9 _2_4___8_|
|_____6___ _2_4__7__ _2_4__7__|1________ ________9 ____5____|__34__78_ _234_____ _2_4__78_|
|12_______ ________9 _______8_|_23______ __34_____ _2_4_____|1_345_7__ _____6___ _2_4__7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 12__5____ 12__5___9|____5_7_9 _____6___ 1__4__7__|___45_7_9 _2_45___9 __3______|
|___4_____ _2__5____ _2__56__9|_______8_ ____5____ __3______|____5_7_9 _2__5___9 1__

## Question 2: Propagating all cells, repeatedly

This is a good beginning, but it's not quite enough. 
As we propagate the constraints, cells that did not use to be singletons may have become singletons.  For eample, in the above example, the center cell has become known to be a 5: we need to make sure that also these singletons are propagated. 

This is why we have written propagate_cell so that it returns the set of newly-singleton cells.  
We need now to write a method `full_propagation` that at the beginning starts with a set of `to_propagate` cells (if it is not specified, then we just take it to consist of all singleton cells).  Then, it picks a cell from the to_propagate set, and propagates from it, adding any newly singleton cell to to_propagate.  Once there are no more cells to be propagated, the method returns. 
If this sounds similar to graph reachability, it is ... because it is!  It is once again the algorithmic pattern of keeping a list of work to be done, then iteratively picking an element from the list, doing it, possibly updating the list of work to be done with new work that has to be done as a result of what we just did, and continuing in this fashion until there is nothing left to do. 
We will let you write this function.  The portion you have to write can be done in three lines of code.

In [13]:
### Exercise: define full propagation

def sudoku_full_propagation(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible.
    @param to_propagate: sets of cells from where to propagate.  If None, propagates
        from all singleton cells. 
    @return: nothing.
    """
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    # This code is the (A) code; will be referenced later.
    # YOUR CODE HERE
    while to_propagate != set():
      element = to_propagate.pop()
      to_propagate.update(self.propagate_cell(element))

Sudoku.full_propagation = sudoku_full_propagation


In [14]:
# Here you can write your own tests if you like. 

# YOUR CODE HERE

In [15]:
### 10 points: Tests for full propagation

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.full_propagation()
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [4], [6], [7], [8], [9], [1], [2]], [[6], [7], [2], [1], [9], [5], [3], [4], [8]], [[1], [9], [8], [3], [4], [2], [5], [6], [7]], [[8], [5], [9], [7], [6], [1], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[9], [6], [1], [5], [3], [7], [2], [8], [4]], [[2], [8], [7], [4], [1], [9], [6], [3], [5]], [[3], [4], [5], [2], [8], [6], [1], [7], [9]]]')
assert_equal(sd, sdd)



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ ___4_____|_____6___ ______7__ _______8_|________9 1________ _2_______|
|_____6___ ______7__ _2_______|1________ ________9 ____5____|__3______ ___4_____ _______8_|
|1________ ________9 _______8_|__3______ ___4_____ _2_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|______7__ _____6___ 1________|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|________9 _____6___ 1________|____5____ __3______ ______7__|_2_______ _______8_ ___4_____|
|_2_______ _______8_ ______7__|___4_____ 1________ ________9|_____6___ __3______

We solved our example problem!  Constraint propagation, iterated, led us to the solution!

## Searching for a solution

Many Sudoku problems can be solved entirely by constraint propagation.  
They are designed to be so: they are designed to be relatively easy, so that humans can solve them while on a lounge chair at the beach -- I know this from personal experience! 

But it is by no means necessary that this is true. 
If we create more complex problems, or less determined problems, constraint propagation no longer suffices. 
As a simple example, let's just blank some cells in the previous problem, and run full propagation again:

In [16]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
sd.show()
sd.full_propagation()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+---------------------

As we see, there are still undetermined values.  We can peek into the detailed state of the solution:


In [17]:
sd.show(details=True)
# Let's save this Sudoku for later.
sd_partially_solved = Sudoku(sd)


+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

What can we do when constraint propagation fails? 
The only thing we can do is make a guess.  We can take one of the cells whose set contains multiple digits, such as cell (2, 0) (starting counting at 0, as in Python), which contains $\{1, 2\}$, and try one of the values, for instance $1$.  
We can see whether assigning to the cell the singleton set $\{1\}$ leads to the solution. 
If not, we try the value $\{2\}$ instead. 
If the Sudoku problem has a solution, one of these two values must work. 

Classically, this way of searching for a solution has been called search with _backtracking._
The backtracking is because we can choose a value, say $1$, and then do a lot of work, propagating the new constraint, making further guesses, and so on and so forth.  If that does not pan out, we must "backtrack" and return to our guess, choosing (in our example) $2$ instead. 

Let us implement search with backtracking.  What we need to do is something like this: 

search():
1. propagate constraints.
1. if solved, hoorrayy!
1. if impossible, raise Unsolvable()
1. if not fully solved, pick a cell with multiple digits possible, and iteratively:
 * Assign one of the possible values to the cell. 
 * Call search() with that value for the cell.
 * If Unsolvable is raised by the search() call, move on to the next value.
 * If all values returned Unsolvable (if we tried them all), then we raise Unsolvable.

So we see that search() is a recursive function.  
From the pseudo-code above, we see that it might be better to pick a cell with few values possible at step 4 above, so as to make our chances of success as good as possible.  For instance, it is much better to choose a cell with set $\{1, 2\}$ than one with set $\{1, 3, 5, 6, 7, 9\}$, as the probability of success is $1/2$ in the first case and $1/6$ in the second case. 
Of course, it may be possible to come up with much better heuristics to guide our search, but this will have to do so far. 

One fine point with the search above is the following.  So far, an object has a self.m matrix, which contains the status of the Sudoku solution. 
We cannot simply pass self.m recursively to search(), because in the course of the search and constraint propagation, self.m will be modified, and there is no easy way to keep track of these modifications. 
Rather, we will write search() as a method, and when we call it, we will:

* First, create a copy of the current object via the Sudoku constructor, so we have a copy we can modify. 
* Second, we assign one of the values to the cell, as above; 
* Third, we will call the search() method of that object. 

Furthermore, when a solution is found, as in the hoorraay! above, we need to somehow return the solution. 
There are two ways of doing this: via standard returns, or by raising an exception. 


In [18]:
def sudoku_done(self):
    """Checks whether an instance of Sudoku is solved."""
    for i in range(9):
        for j in range(9):
            if len(self.m[i][j]) > 1:
                return False
    return True

Sudoku.done = sudoku_done


def sudoku_search(self, new_cell=None):
    """Tries to solve a Sudoku instance."""
    to_propagate = None if new_cell is None else {new_cell}
    self.full_propagation(to_propagate=to_propagate)
    if self.done():
        return self # We are a solution
    # We need to search.  Picks a cell with as few candidates as possible.
    candidates = [(len(self.m[i][j]), i, j)
                   for i in range(9) for j in range(9) if len(self.m[i][j]) > 1]
    _, i, j = min(candidates)
    values = self.m[i][j]
    # values contains the list of values we need to try for cell i, j.
    # print("Searching values", values, "for cell", i, j)
    for x in values:
        # print("Trying value", x)
        sd = Sudoku(self)
        sd.m[i][j] = {x}
        try:
            # If we find a solution, we return it.
            return sd.search(new_cell=(i, j))
        except Unsolvable:
            # Go to next value.
            pass
    # All values have been tried, apparently with no success.
    raise Unsolvable()

Sudoku.search = sudoku_search


def sudoku_solve(self, do_print=True):
    """Wrapper function, calls self and shows the solution if any."""
    try:
        r = self.search()
        if do_print:
            print("We found a solution:")
            r.show()
            return r
    except Unsolvable:
        if do_print:
            print("The problem has no solutions")

Sudoku.solve = sudoku_solve


Let us try this on our previous Sudoku problem that was not solvable via constraint propagation alone.

In [19]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
_ = sd.solve()


We found a solution:
+---+---+---+
|531|678|942|
|674|295|318|
|298|341|567|
+---+---+---+
|859|167|423|
|426|853|791|
|713|924|856|
+---+---+---+
|165|739|284|
|987|412|635|
|342|586|179|
+---+---+---+


It works, search with constraint propagation solved the Sudoku puzzle!

## The choice - constraint propagation - recursion paradigm.

We have learned a general strategy for solving difficult problems.  The strategy can be summarized thus: **choice - constraint propagation - recursion.** 

In the _choice_ step, we make one guess from a set of possible guesses.  If we want our search for a solution to be exhaustive, as in the above Sudoku example, we ensure that we try iteratively all choices from a set of choices chosen so that at least one of them must succeed.  In the above example, we know that at least one of the digit values must be the true one, hence our search is exhaustive.  In other cases, we can trade off exhaustiveness for efficiency, and we may try only a few choices, guided perhaps by an heuristic. 

The _constraint propagation_ step propagates the consequences of the choice to the problem.  Each choice thus gives rise to a new problem, which is a little bit simpler than the original one as some of the possible choices, that is, some of its complexity, has been removed.  In the Sudoku case, the new problem has less indetermination, as at least one more of its cells has a known digit in it. 

The problems resulting from _constraint propagation_, while simpler, may not be solved yet.  Hence, we _recur_, calling the solution procedure on them as well.  As these problems are simpler (they contain fewer choices), eventually the recursion must reach a point where no more choice is possible, and whether constraint propagation should yield a completely defined problem, one of which it is possible to say whether it is solvable or not with a trivial test.  This forms the base case for the recursion. 

This solution strategy applies very generally, to problems well beyond Sudoku.

## Part 2: Digits must go somewhere

If you have played Sudoku before, you might have found the way we solved Sudoku puzzles a bit odd. 
The constraint we encoded is: 

> If a digit appears in a cell, it cannot appear anywhere else on the same row, column, or 3x3 block as the cell. 

This _is_ a rule of Sudoku.  Normally, however, we hear Sudoku described in a different way:

> Every column, row, and 3x3 block should contain all the 1...9 digits exactly once.

There are two questions.  The first is: are the two definitions equivalent? 
Well, no; the first definition does not say what the digits are (e.g., does not rule out 0).  But in our Sudoku representation, we _start_ by saying that every cell can contain only one of 1...9.  If every row (or column, or 3x3 block) cannot contain more than one repetition of each digit, and if there are 9 digits and 9 cells in the row (or column, or block), then clearly every digit must appear exactly once in the row (or column, or block).  So once the set of digits is specified, the two definitions are equivalent. 

The second question is: but still, what happens to the method we usually employ to solve Sudoku? 
I generally don't solve Sudoku puzzles by focusing on one cell at a time, and thinking: is it the case that this call can contain only one digit? 
This is the strategy employed by the solver above.  But it is not the strategy I normally use. 
I generally solve Sudoku puzzles by looking at a block (or row, or column), and thinking: let's consider the digit $k$ ($1 \leq k \leq 9$).  Where can it go in the block?  And if I find that the digit can go in one block cell only, I write it there.  
Does the solver work even without this "where can it go" strategy?  And can we make it follow it? 

The solver works even without the "where can it go" strategy because it exaustively tries all possibilities.  This means the solver works without the strategy; it does not say that the solver works _well_ without the strategy. 

We can certainly implement the _where can it go_ strategy, as part of constraint propagation; it would make our solver more efficient. 


## Question 3: A better `full_propagation` method
### Not a real question; just copy some previous code into a new method.
There is a subtle point in applying the _where can it go_ heuristics. 

Before, when our only constraint was the uniqueness in each row, column, and block, we needed to propagate only from cells that hold a singleton value. 
If a cell held a non-singleton set of digits, such as $\{2, 5\}$, no values could be ruled out as a consequence of this on the same row, column, or block. 

The _where can it go_ heuristic, instead, benefits from knowing that in a cell, the set of values went for instance from $\{2, 3, 5\}$ to $\{2, 5\}$: by ruling out the possibility of a $3$ in this cell, it may be possibe to deduct that the digit $3$ can appear in only one (other) place in the block, and place it there. 

Thus, we modify the `full_propagation` method.  The method does:
* Repeat:
  * first does propagation as before, based on singletons; 
  * then, it applies the _where can it go_ heuristic on the whole Sudoku board. 
* until there is nothing more that can be propagated. 

Thus, we replace the `full_propagation` method previously defined with this new one, where the (A) block of code is what you previously wrote in `full_propagation`.
You don't need to write new code here: just copy your solution for `full_propagation` into the (A) block below. 

In [20]:
### Exercise: define full propagation with where can it go

def sudoku_full_propagation_with_where_can_it_go(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible."""
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    while len(to_propagate) > 0:
        # Here is your previous solution code from (A) in full_propagation.
        # Please copy it below. No change is required. 
        # YOUR CODE HERE
        while to_propagate != set():
          element = to_propagate.pop()
          to_propagate.update(self.propagate_cell(element))
        # Now we check whether there is any other propagation that we can
        # get from the where can it go rule.
        to_propagate = self.where_can_it_go()



## Question 4: Implement the `occurs_once_in_sets` helper

To implement the `where_can_it_go` method, let us write a helper function, or better, let's have you write it.  Given a sequence of sets $S_1, S_2, \ldots, S_n$, we want to obtain the set of elements that appear in _exactly one_ of the sets (that is, they appear in one set, and _only_ in one set).   Mathematically, we can write this as 
$$
(S_1 \setminus (S_2 \cup \cdots \cup S_n)) \cup (S_2 \setminus (S_1 \cup S_3 \cup \cdots \cup S_n)) \cup \cdots \cup
(S_n \setminus (S_1 \cup \cdots \cup S_{n-1}))
$$
even though that's certainly not the easiest way to compute it!
The problem can be solved with the help of [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict!) to count the occurrences, and is 5 lines long.
Of course, other solutions are possible as well. 


In [21]:
### Exercise: define helper function to check once-only occurrence

from collections import defaultdict

def occurs_once_in_sets(set_sequence):
    """Returns the elements that occur only once in the sequence of sets set_sequence.
    The elements are returned as a set."""
    # YOUR CODE HERE
    unique = defaultdict(int)
    for sequence in set_sequence:
        for item in sequence:
          unique[item] +=1
    return set([item for item in unique.keys() if unique[item] == 1])


In [22]:
# Here you can write your own tests if you like. 

# YOUR CODE HERE

Let us test it.

In [23]:
### 10 points: Tests for once-only

from nose.tools import assert_equal

assert_equal(occurs_once_in_sets([{1, 2}, {2, 3}]), {1, 3})
assert_equal(occurs_once_in_sets([]), set())
assert_equal(occurs_once_in_sets([{2, 3, 4}]), {2, 3, 4})
assert_equal(occurs_once_in_sets([set()]), set())
assert_equal(occurs_once_in_sets([{2, 3, 4, 5, 6}, {5, 6, 7, 8}, {5, 6, 7}, {4, 6, 7}]), {2, 3, 8})



## Question 5: Implement _where can it go_. 

We are now ready to write -- or better, to have you write -- the _where can it go_ method.  
The method is global: it examines all rows, all columns, and all blocks.  
If it finds that in a row (or column, or block), a value can fit in only one cell, and that cell is not currently a singleton (for otherwise there is nothing to be done), it sets the value in the cell, and it adds the cell to the newly_singleton set that is returned, just as in propagate_cell. 
The portion of method that you need to write is about two dozen lines of code long.

In [24]:
### Exercise: write where_can_it_go

def sudoku_where_can_it_go(self):
    """Sets some cell values according to the where can it go
    heuristics, by examining all rows, colums, and blocks."""
    newly_singleton = set()

    # YOUR CODE HERE
    for i in range(len(self.m)):
      for j in range(len(self.m[0])):
        x = self.m[i][j]
        if len(x) > 1:
          col,row,cell = [],[],[]
          for jj in range(9): 
            row.append(self.m[i][jj])
          for ii in range(9): 
            col.append(self.m[ii][j])
          for ii in range(i//3*3,i//3*3+3):
            for jj in range(j//3*3,j//3*3+3):
              cell.append(self.m[ii][jj])
          unique_col = occurs_once_in_sets(col)
          unique_row = occurs_once_in_sets(row)
          unique_cell = occurs_once_in_sets(cell)
          for vals in x:
            if vals in unique_col or vals in unique_row or vals in unique_cell:
              newly_singleton.add((i,j))
              self.m[i][j] = {vals}
              break
            # if vals in unique_row:
            #   newly_singleton.add((i,j))
            #   self.m[i][j] = {vals}
            #   break
            # if vals in unique_cell:
            #   newly_singleton.add((i,j))
            #   self.m[i][j] = {vals}
            #   break
      

    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.where_can_it_go = sudoku_where_can_it_go


In [25]:
# Here you can write your own tests if you like. 

import random
for _ in range(10000):
    nums=[a for a in range(random.randint(1,20))]
    setList=[set() for _ in range(random.randint(1,10))]
    L1=len(setList)==1
    inOne=set()
    while len(nums)>0:
        num=nums.pop()
        if random.random()<.25:
            random.choice(setList).add(num)
            inOne.add(num)
        else:
            setList=[a|{num} for a in setList]
            if L1:
                inOne.add(num)
    try:
        assert_equal(occurs_once_in_sets(setList),inOne)
    except:
        print('failed, printing information:')
        print('set_sequence:  ',setList)
        print('your answer:   ',occurs_once_in_sets(setList))
        print('correct answer:',inOne)
        assert_equal(occurs_once_in_sets(setList),inOne)
# YOUR CODE HERE

Let us test it.  We cannot test this code in one iteration only, since its result may depend on the order in which you apply the method to rows and columns. 
Rather, we apply the method until it can determine no more cell values.

In [26]:
### Tests for where can it go

sd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [1, 2, 6], [7], [1, 2, 6, 8], [4, 8, 9], [1, 2, 4], [2, 8]], [[6], [1, 4, 7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3, 4, 8], [1, 2, 4], [2, 7, 8]], [[1, 2], [9], [8], [1, 2, 3], [4], [1, 2], [3, 5], [6], [2, 7]], [[8], [1, 5], [1, 5, 9], [1, 7, 9], [6], [1, 4, 7, 9], [4, 5], [2, 4, 5], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1, 5], [1, 3, 5, 9], [1, 9], [2], [1, 4, 9], [4, 5, 8], [4, 5], [6]], [[1, 9], [6], [1, 5, 7, 9], [5, 7, 9], [3], [7, 9], [2], [8], [4]], [[2, 9], [7, 8], [2, 7, 9], [4], [1], [2, 7, 9], [6], [3], [5]], [[2, 3], [4, 5], [2, 3, 4, 5], [2, 5, 6], [8], [2, 6], [1], [7], [9]]]')
print("Original:")
sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
print(new_singletons)
assert_equal(new_singletons,
             {(3, 2), (2, 6), (7, 1), (5, 6), (2, 8), (8, 0), (0, 5), (1, 6),
              (2, 3), (3, 7), (0, 3), (5, 1), (0, 8), (8, 5), (5, 3), (5, 5),
              (8, 1), (5, 7), (3, 1), (0, 6), (1, 8), (3, 6), (5, 2), (1, 1)})
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)

sd = Sudoku([
    '___26_7_1',
    '68__7____',
    '1____45__',
    '82_1___4_',
    '__46_2___',
    '_5___3_28',
    '___3___74',
    '_4__5__36',
    '7_3_18___'
])
print("Another Original:")
sd.show(details=True)
print("Propagate once:")
sd.propagate_all_cells_once()
# sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[4], [3], [5], [2], [6], [9], [7], [8], [1]], [[6], [8], [2], [5], [7], [1], [4], [9], [3]], [[1], [9], [7], [8], [3], [4], [5], [6], [2]], [[8], [2], [6], [1], [9], [5], [3], [4], [7]], [[3], [7], [4], [6], [8], [2], [9], [1], [5]], [[9], [5], [1], [7], [4], [3], [6], [2], [8]], [[5], [1], [1, 2, 5, 6, 8, 9], [3], [2], [6], [1, 2, 8, 9], [7], [4]], [[2], [4], [1, 2, 8, 9], [9], [5], [7], [1, 2, 8, 9], [3], [6]], [[7], [6], [3], [4], [1], [8], [2], [5], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)



Original:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___

Let us try it now on a real probem. Note from before that this Sudoku instance could not be solved via propagate_cells alone:

In [27]:
sd = Sudoku(sd_partially_solved)
newly_singleton = sd.where_can_it_go()
print("Newly singleton:", newly_singleton)
print("Resulting Sudoku:")
sd.show(details=True)


Newly singleton: {(2, 6), (7, 1), (5, 6), (2, 8), (5, 7), (8, 0), (8, 5), (0, 6), (0, 5), (3, 7), (5, 2)}
Resulting Sudoku:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ _______8_|________9 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ __3______|1_______9 _2_______ 1__4____9|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3____

As we can see, the heuristics led to substantial progress.   Let us incorporate it in the Sudoku solver. 

In [28]:
Sudoku.full_propagation = sudoku_full_propagation_with_where_can_it_go


Let us try again to solve a Sudoku example which, as we saw before, could not be solved by constrain propagation only (without using the _where can it go_ heuristics).  Can we solve it now via constraint propagation?

In [29]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
print("Initial:")
sd.show()
sd.full_propagation()
print("After full propagation with where can it go:")
sd.show()


Initial:
+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
After full propagation with where can it go:
+---+---+---+
|53.|678|9.2|
|67.|.95|3.8|
|.98|34.|567|
+---+---+---+
|859|.6.|423|
|426|853|791|
|713|924|856|
+---+---+---+
|.6.|.3.|284|
|.8.|41.|635|
|34.|.86|179|
+---+---+---+


No!  We still cannot! But if we compare the above with the previous attempt, we see that the heuristic led to much more progress; very few positions still remain to be determined via search. 

## Question 6: Solving some problems from example sites

Let us see how long it takes us to solve examples found around the Web. 
We consider a few from [this site](https://dingo.sbs.arizona.edu/~sandiway/sudoku/examples.html).
You should be able to complete all of these tests in a short amount of time. 

In [30]:
import time


### Daily Telegraph January 19th "Diabolical"



In [31]:
# 5 points: You need to do this in less than 5. 

sd = Sudoku([
    '_2_6_8___',
    '58___97__',
    '____4____',
    '37____5__',
    '6_______4',
    '__8____13',
    '____2____',
    '__98___36',
    '___3_6_9_'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")

assert elapsed < 5

We found a solution:
+---+---+---+
|123|678|945|
|584|239|761|
|967|145|328|
+---+---+---+
|372|461|589|
|691|583|274|
|458|792|613|
+---+---+---+
|836|924|157|
|219|857|436|
|745|316|892|
+---+---+---+
Solved in 0.010692119598388672 seconds


### Vegard Hanssen puzzle 2155141

In [32]:
# 5 points: you need to do this in less than 5 seconds.

sd = Sudoku([
    '___6__4__',
    '7____36__',
    '____91_8_',
    '_________',
    '_5_18___3',
    '___3_6_45',
    '_4_2___6_',
    '9_3______',
    '_2____1__'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")
assert elapsed < 5

We found a solution:
+---+---+---+
|581|672|439|
|792|843|651|
|364|591|782|
+---+---+---+
|438|957|216|
|256|184|973|
|179|326|845|
+---+---+---+
|845|219|367|
|913|768|524|
|627|435|198|
+---+---+---+
Solved in 0.04815340042114258 seconds


### A supposedly even harder one

[source](http://www.sudokuwiki.org/Weekly_Sudoku.asp?puz=28)

In [33]:
# 5 points: you need to do this in less than 10 seconds. 

sd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")
assert elapsed < 10

We found a solution:
+---+---+---+
|625|178|943|
|948|326|157|
|371|945|862|
+---+---+---+
|257|619|384|
|463|587|291|
|189|432|576|
+---+---+---+
|792|863|415|
|516|294|738|
|834|751|629|
+---+---+---+
Solved in 1.0242981910705566 seconds


## Trying puzzles in bulk

Let us try the puzzles found at [https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt](https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt); apparently lines 517 and 6361 are very hard). 

In [34]:
import requests

r = requests.get("https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt")
puzzles = r.text.split()


Let us convert these puzzles to our format.

In [35]:
def convert_to_our_format(s):
    t = s.replace('0', '_')
    r = []
    for i in range(9):
        r.append(t[i * 9: (i + 1) * 9])
    return r


You need to solve these tests efficiently.

In [36]:
# 5 points: you need to solve the first 1000 Sudokus in less than 30 seconds.

t = 0
max_d = 0.
max_i = None
t = time.time()
for i, s in enumerate(puzzles[:1000]):
    p = convert_to_our_format(puzzles[i])
    sd = Sudoku(p)
    sd.solve(do_print=True)
elapsed = time.time() - t
print("It took you", elapsed, "to solve the first 1000 Sudokus.")
assert elapsed < 30

Streaming output truncated to the last 5000 lines.
+---+---+---+
We found a solution:
+---+---+---+
|432|165|978|
|681|379|254|
|597|842|613|
+---+---+---+
|258|436|791|
|769|581|342|
|143|927|586|
+---+---+---+
|374|698|125|
|916|254|837|
|825|713|469|
+---+---+---+
We found a solution:
+---+---+---+
|621|543|879|
|739|182|645|
|548|697|321|
+---+---+---+
|982|371|564|
|364|259|718|
|175|468|293|
+---+---+---+
|497|836|152|
|253|714|986|
|816|925|437|
+---+---+---+
We found a solution:
+---+---+---+
|621|534|879|
|739|182|645|
|548|697|321|
+---+---+---+
|982|371|564|
|463|259|718|
|175|468|293|
+---+---+---+
|397|846|152|
|254|713|986|
|816|925|437|
+---+---+---+
We found a solution:
+---+---+---+
|354|726|981|
|126|598|734|
|789|134|265|
+---+---+---+
|478|253|619|
|932|861|547|
|615|479|328|
+---+---+---+
|867|912|453|
|543|687|192|
|291|345|876|
+---+---+---+
We found a solution:
+---+---+---+
|425|736|981|
|136|298|745|
|789|145|263|
+---+---+---+
|578|324|619|
|942|861|357|
|613